In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "passpass01"
hostname= "nv-desktop-services.apporto.com"
port = 31649
database_name = "AAC"
collection_name = "animals"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, hostname, port, database_name)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date


app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}),
        html.Div([
            html.H4('By: Dawson Kennedy')
        ], style={'text-align': 'left', 'margin-left': '10%'})
    ]),
    html.Hr(),
    
    #Add interactive filtering options.

    html.Div([
        dcc.RadioItems(
            id="filter-type",
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block'}
        )
            
    ]),
    
    #Set up the features for your interactive data table to make it user-friendly for your client
    
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #interactive data table to make it user-friendly for your client
        page_size=10,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[0],

    ),
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # If user selects 'Reset', show all data
    if filter_type == 'Reset':
        print("Reset selected")
        filtered_df = df
    # Otherwise, filter data based on the selected rescue type
    else:
        if filter_type == 'Water Rescue':
            query = {"$and": [{"animal_type": "Dog"}, 
                              {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
                              {"sex_upon_outcome": "Intact Female"}]}
        elif filter_type == 'Mountain or Wilderness Rescue':
            query = {"$and": [{"animal_type": "Dog"}, 
                              {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                                                "Siberian Husky", "Rottweiler"]}},
                              {"sex_upon_outcome": "Intact Male"}]}
        elif filter_type == 'Disaster or Individual Tracking':
            query = {"$and": [{"animal_type": "Dog"}, 
                              {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}},
                              {"sex_upon_outcome": "Intact Male"}]}
        
        filtered_df = pd.DataFrame(list(db.read(query)))
        if '_id' in filtered_df.columns:
            filtered_df.drop(columns=['_id'], inplace=True)
    
    # Debugging line to check filtered_df content
    print("Filtered DataFrame:", filtered_df)
            
    return filtered_df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available to display.")
    
    # Convert the viewData into a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Create a pie chart using Plotly Express
    fig = px.pie(dff, names='breed', title='Breed Distribution in Selected Data')
    
    return [
        dcc.Graph(
            figure=fig
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return[]
    return[{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    }for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available for map display.")
    # Convert the viewData into a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Check if DataFrame is empty
    if dff.empty or len(dff.columns) < 14:
        return html.Div("Insufficient data for map display.")
    
    # Ensure index is valid
    row = index[0] if index else 0
    
    # Debugging output
    print("View Data Columns:", dff.columns)
    print("Selected Row Data:", dff.iloc[row])
    
    # Check that the row and column exist before accessing
    try:
        lat, lon = dff.iloc[row, 13], dff.iloc[row, 14]
    except IndexError as e:
        print(f"IndexError: {e}")
        return html.Div("Error accessing map data.")
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:17862/
Reset selected
Filtered DataFrame:       rec_num age_upon_outcome animal_id animal_type  \
0           2           1 year   A725717         Cat   
1           1          3 years   A746874         Cat   
2           3          2 years   A716330         Dog   
3           5          2 years   A691584         Dog   
4           4         7 months   A733653         Cat   
...       ...              ...       ...         ...   
9994     9998         2 months   A697165         Dog   
9995     9999         2 months   A665304         Dog   
9996     9997          5 years   A745429         Cat   
9997     9994          2 years   A713640         Cat   
9998     9995          3 years   A733093         Dog   

                                   breed         color date_of_birth  \
0                 Domestic Shorthair Mix  Silver Tabby    2015-05-02   
1                 Domestic Shorthair Mix   Black/White    2014-04-10   
2                Chihuahua Short